<a href="https://colab.research.google.com/github/Sanjeev-coder/Identifying_Credit_card_Bank_defaulter-s_Customer/blob/main/Credit_card_Banks_defaulter_customers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install apache_beam[interactive]

In [4]:
import apache_beam as beam


p = beam.Pipeline()

def calculate_points(element):

  customer_id, first_name, last_name, realtionship_id, card_type, max_limit, spent, cash_withdrawn,payment_cleared,payment_date = element.split(',')
  #[CT28383,Miyako,Burns,R_7488,Issuers,500,490,38,101,30-01-2018]

  spent = int(spent)    # spent = 490
  payment_cleared = int(payment_cleared)   #payment_cleared = 101
  max_limit = int(max_limit)               # max_limit = 500

  key_name = customer_id + ', ' + first_name + ' ' + last_name     # key_name = CT28383,Miyako Burns
  defaulter_points = 0

  # payment_cleared is less than 70% of spent - give 1 point
  if payment_cleared < (spent * 0.7):
     defaulter_points += 1                                                # defaulter_points =  1

  # spend is = 100% of max limit and any amount of payment is pending
  if (spent == max_limit) and (payment_cleared < spent):
     defaulter_points += 1                                                # defaulter_points =  2

  if (spent == max_limit) and (payment_cleared < (spent*0.7)):
     defaulter_points += 1                                                # defaulter_points = 3

  return key_name, defaulter_points                                     # {CT28383,Miyako Burns  3}

def format_result(sum_pair):
  key_name, points = sum_pair
  return str(key_name) + ', ' + str(points) + ' fraud_points'


card_defaulter = (
                  p
                  | 'Read credit card data' >> beam.io.ReadFromText('/content/cards.txt',skip_header_lines=1)
                  | 'Calculate defaulter points' >> beam.Map(calculate_points)
                  | 'Combine points for defaulters' >> beam.CombinePerKey(sum)                            # key--> CT28383,Miyako Burns   value --> 6
                  | 'Filter card defaulters' >> beam.Filter(lambda element: element[1] > 0)
                  | 'Format output' >> beam.Map(format_result)                                            # CT28383,Miyako Burns,6 fraud_points
                  | "print the output " >>beam.Map(print)

                  )



p.run()

CT28383, Miyako Burns, 3 fraud_points
CT74474, Nanaho Brennan, 3 fraud_points
CT66322, Chris Bruce, 1 fraud_points
CT65528, Bonnie Barlow, 2 fraud_points
CT84463, Isaac Bowman, 4 fraud_points
CT12838, Isidore Albert, 5 fraud_points
CT96185, Danielle Bradshaw, 3 fraud_points
CT74827, Hanna Boyer, 3 fraud_points
CT98239, Sayuri Case, 4 fraud_points
CT57141, Kaori Brown, 2 fraud_points
CT85915, Dean Becker, 5 fraud_points
CT86894, Miwa Carter, 5 fraud_points
CT33583, Odette Case, 4 fraud_points
CT64751, Youko Bryant, 3 fraud_points
CT14528, Omar Acevedo, 6 fraud_points
CT98422, Nicole Caldwell, 2 fraud_points
CT66224, Ivan Blanchard, 2 fraud_points
CT91012, Jerry Barron, 4 fraud_points
CT64089, Fay Beard, 4 fraud_points
CT56854, Michael Cannon, 4 fraud_points
CT46595, Marco Byers, 5 fraud_points
CT47648, Kyle Butler, 4 fraud_points
CT79129, Kaori Campos, 4 fraud_points
CT13659, Omar Alston, 2 fraud_points
CT46586, Walter Camacho, 5 fraud_points
CT77821, Miwa Bird, 3 fraud_points
CT67931, 